In [ ]:
import kwant
import numpy as np 
import matplotlib.pyplot as plt 
from scipy.linalg import eig
import qsymm

sigma = np.array([[[0, 1], [1, 0]],
                  [[0, 1j], [-1j, 0]],
                  [[1, 0], [0, -1]],
                  [[1, 0],[0, 1]]])


lat_h = kwant.lattice.chain(norbs=1)
bulk_h = kwant.Builder(symmetry=kwant.lattice.TranslationalSymmetry([2]))

lat_a = kwant.lattice.chain(norbs=1)
bulk_a = kwant.Builder(symmetry=kwant.lattice.TranslationalSymmetry([2]))

def onsite_h(site):
    return 0

def hopping_hintra(site1, site2, t1):
    return t1

def hopping_hinter(site1, site2, t2):
    return t2

bulk_h[lat_h(0)] = onsite_h
bulk_h[lat_h(1)] = onsite_h
bulk_h[lat_h(0),lat_h(1)] = hopping_hintra
bulk_h[lat_h(1),lat_h(2)] = hopping_hinter

def onsite_a(site):
    return 0 

def hopping_aintra(site1, site2, v1):
    return 1j*v1

bulk_a[lat_a(0)] = onsite_a
bulk_a[lat_a(1)] = onsite_a
bulk_a[lat_a(0),lat_a(1)] = hopping_aintra
bulk_a[lat_a(1),lat_a(2)] = 0

finite_h = kwant.Builder()
finite_a = kwant.Builder()

L = 150

def shape(site):
    pos=site.pos
    return -L/2<pos[0]<=L/2

finite_h.fill(bulk_h, shape, start=[0]);
finite_a.fill(bulk_a, shape, start=[0]);

finite_h = finite_h.finalized()
finite_a = finite_a.finalized()

pol = []

x = np.diag([s.pos[0] for s in finite_h.sites])
x = x*(1/L)

ts = np.arange(0.5, 1.5, 0.01)

for t1 in ts: 
    
    aux = t1

    params_h = dict(t1=aux, t2=1)
    ham_h = finite_h.hamiltonian_submatrix(params = params_h)

    params_a = dict(v1=3)
    ham_a = finite_a.hamiltonian_submatrix(params = params_a)

    ham_tot = ham_h + 1j*ham_a  
    
    evals, evecsL, evecsR = eig(ham_tot, left=True, right=True) 
    #evalsR, evecsR = eig(ham_tot, left=False, right=True)
    
    
    evecsR2=(1/np.diag(evecsL.conj().T@evecsR))*evecsR
    
    P = 1 - evecsL.conj().T@x@evecsR2
    
    K = np.diag(P)
    
    plt.scatter(t1*np.ones(K.shape), K.real)

    

In [ ]:
print(evecsL[:,34].conj().T@evecsR[:,34])

In [ ]:
evecsR2=(1/np.diag(evecsL.conj().T@evecsR))*evecsR

In [ ]:
print(evecsL[:,34].conj().T@evecsR2[:,34])

In [2]:
import qsymm
import numpy as np

qsymm.__version__
qsymm??

In [3]:
qsymm.groups.PointGroupElement(R=np.eye(2), transpose=True)

1

In [6]:
from importlib.util import module_from_spec, spec_from_file_location